In [1]:
# Cell 1: Setup imports
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Thêm đường dẫn gốc của dự án vào sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import TradingEngine và components
from src.core.trading_engine import TradingEngine
from src.strategies.sma_crossover import SMACrossoverStrategy
from src.strategies.rsi_strategy import RSIStrategy
from src.strategies.macd_strategy import MACDStrategy
from src.utils.config_manager import ConfigManager
from src.data.data_manager import DataManager

# Import optimization components
from src.optimization.optimizer import StrategyOptimizer
from src.optimization.parameter_grid import ParameterGrid

# Import optimization visualization
from optimization_visualization import (
    plot_optimization_results, 
    plot_parameter_heatmap, 
    plot_optimization_progress,
    create_optimization_report
)

# Import visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

print("Import thành công!")

Import thành công!


In [2]:
# Cell 2: Khởi tạo TradingEngine
config_path = "../../config/config.yaml"
engine = TradingEngine(config_path)

print(f"TradingEngine đã được khởi tạo với vốn ban đầu: ${engine.cash:,.2f}")
print(f"Config loaded: {engine.config.get('trading.symbols')}")
print(f"Config loaded: {engine.config.get('data.source')}")

2025-08-02 15:11:57.333 | INFO     | src.utils.config_manager:__init__:31 - Configuration loaded from ../../config/config.yaml
2025-08-02 15:11:57.334 | INFO     | src.risk.risk_manager:__init__:49 - Risk manager initialized
2025-08-02 15:11:57.334 | INFO     | src.core.trading_engine:__init__:77 - Trading engine initialized with $100,000.00 initial capital


TradingEngine đã được khởi tạo với vốn ban đầu: $100,000.00
Config loaded: ['Bitstamp:BTCUSD']
Config loaded: tradingview


In [3]:
# Cell 3: Chuẩn bị data
# Sử dụng data có sẵn hoặc lấy từ DataManager
symbols = engine.config.get("trading.symbols", ["AAPL"])
start_date = engine.config.get("data.start_date", "2023-01-01")
end_date = engine.config.get("data.end_date", "2023-12-31")


# Lấy data từ DataManager
data_manager = DataManager(engine.config)
historical_data = data_manager.get_historical_data(
    symbols=symbols,
    start_date=start_date,
    end_date=end_date,
    interval=engine.config.get("data.interval", "1d"),
    n_bars=engine.config.get("data.n_bars", 1000)
)

print(f"Lấy data cho symbols: {symbols}")
# print(f"Period: {start_date} đến {end_date}")
print(f"Data shape: {historical_data.shape}")
print(f"Data columns: {historical_data.columns.tolist()}")
print(
    f"Date range: {historical_data.index.min()} \
        đến {historical_data.index.max()} \
            interval: {engine.config.get('data.interval', '1d')}"
)

2025-08-02 15:12:00.856 | INFO     | src.data.data_manager:get_historical_data:64 - Fetching historical OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2025-07-31
2025-08-02 15:12:01.223 | INFO     | src.data.data_manager:_fetch_tradingview_ohlcv_data:159 - Successfully fetched OHLCV data for 1 symbols
2025-08-02 15:12:01.226 | INFO     | src.data.data_manager:get_historical_data:97 - OHLCV data cached successfully


Lấy data cho symbols: ['Bitstamp:BTCUSD']
Data shape: (100, 5)
Data columns: [('Bitstamp:BTCUSD', 'open'), ('Bitstamp:BTCUSD', 'high'), ('Bitstamp:BTCUSD', 'low'), ('Bitstamp:BTCUSD', 'close'), ('Bitstamp:BTCUSD', 'volume')]
Date range: 2025-07-17 03:00:00         đến 2025-08-02 15:00:00             interval: 4h


In [4]:
# Cell 4: Khởi tạo Strategy Optimizer
optimizer = StrategyOptimizer(engine.config)
parameter_grid = ParameterGrid()

print("Strategy Optimizer đã được khởi tạo")
print("Available strategy types for optimization:")
for strategy_type in parameter_grid.grids.keys():
    print(f"  - {strategy_type}")

2025-08-02 15:12:05.418 | INFO     | src.risk.risk_manager:__init__:49 - Risk manager initialized
2025-08-02 15:12:05.419 | INFO     | src.backtesting.backtest_engine:__init__:38 - Backtest engine initialized
2025-08-02 15:12:05.419 | INFO     | src.optimization.optimizer:__init__:42 - Strategy optimizer initialized


Strategy Optimizer đã được khởi tạo
Available strategy types for optimization:
  - sma_crossover
  - rsi
  - macd
  - bollinger_bands


In [5]:
# Cell 5: Optimize parameters cho từng strategy
print("=== BẮT ĐẦU OPTIMIZE PARAMETERS ===")

# Cấu hình optimization
optimization_start_date = "2023-01-01"
optimization_end_date = "2023-06-30"  # Sử dụng 6 tháng đầu để optimize
backtest_start_date = "2023-07-01"    # Sử dụng 6 tháng cuối để backtest
backtest_end_date = "2023-12-31"

optimization_metric = 'sharpe_ratio'  # Có thể thay đổi: 'total_return', 'profit_factor', 'max_drawdown'
max_combinations_per_strategy = 50    # Giới hạn số combinations để test

# Optimize cho từng strategy
strategy_types = ['sma_crossover', 'rsi', 'macd']
optimization_results = {}

for strategy_type in strategy_types:
    print(f"\n--- Optimizing {strategy_type.upper()} ---")
    
    try:
        # Chạy optimization
        result = optimizer.optimize_strategy(
            strategy_type=strategy_type,
            start_date=optimization_start_date,
            end_date=optimization_end_date,
            optimization_metric=optimization_metric,
            max_combinations=max_combinations_per_strategy,
            use_parallel=True
        )
        
        if result:
            optimization_results[strategy_type] = result
            best_params = optimizer.best_parameters[strategy_type]
            print(f"✅ Optimization completed for {strategy_type}")
            print(f"Best parameters: {best_params}")
            print(f"Best {optimization_metric}: {result.get('best_metric_value', 'N/A')}")
        else:
            print(f"❌ Optimization failed for {strategy_type}")
            
    except Exception as e:
        print(f"❌ Error optimizing {strategy_type}: {str(e)}")

2025-08-02 15:12:33.760 | INFO     | src.optimization.optimizer:optimize_strategy:65 - Starting optimization for sma_crossover
2025-08-02 15:12:33.762 | INFO     | src.optimization.parameter_grid:get_parameter_combinations:67 - Generated 48 parameter combinations for sma_crossover


=== BẮT ĐẦU OPTIMIZE PARAMETERS ===

--- Optimizing SMA_CROSSOVER ---


2025-08-02 15:12:38.058 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: sma_crossover
2025-08-02 15:12:38.058 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: sma_crossover
2025-08-02 15:12:38.058 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: sma_crossover
2025-08-02 15:12:38.058 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: sma_crossover
2025-08-02 15:12:38.058 | INFO     | src.strategies.sma_crossover:__init__:48 - SMA Crossover Strategy initialized: 5/60
2025-08-02 15:12:38.058 | INFO     | src.strategies.sma_crossover:__init__:48 - SMA Crossover Strategy initialized: 5/50
2025-08-02 15:12:38.058 | INFO     | src.strategies.sma_crossover:__init__:48 - SMA Crossover Strategy initialized: 5/40
2025-08-02 15:12:38.058 | INFO     | src.strategies.sma_crossover:__init__:48 - SMA Crossover Strategy initialized: 5/30
2025-08-02 15:12:38.058 | INFO     | src.backtesting

✅ Optimization completed for sma_crossover
Best parameters: {'parameters': {'short_window': 10, 'long_window': 60}, 'metrics': {'total_return': 0.01583959644997357, 'annualized_return': 0.05965229884864942, 'sharpe_ratio': 0.7266163316764024, 'max_drawdown': -0.012492581237808715, 'volatility': 0.028069029884101097, 'win_rate': 0.0, 'profit_factor': 0.0, 'total_trades': 0, 'final_portfolio_value': 101583.95964499735, 'current_drawdown': -0.011239262286793294, 'portfolio_volatility': 0.028069029884101097}, 'rank': 1}
Best sharpe_ratio: N/A

--- Optimizing RSI ---


2025-08-02 15:13:09.732 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: rsi
2025-08-02 15:13:09.732 | INFO     | src.strategies.rsi_strategy:__init__:56 - RSI Strategy initialized: period=20, overbought=70, oversold=30
2025-08-02 15:13:09.732 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: rsi
2025-08-02 15:13:09.732 | INFO     | src.backtesting.backtest_engine:run_backtest:53 - Starting backtest from 2023-01-01 to 2023-06-30
2025-08-02 15:13:09.732 | INFO     | src.data.data_manager:get_historical_data_standardized:589 - Fetching standardized OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2023-06-30
2025-08-02 15:13:09.732 | INFO     | src.strategies.rsi_strategy:__init__:56 - RSI Strategy initialized: period=10, overbought=70, oversold=30
2025-08-02 15:13:09.732 | INFO     | src.data.data_manager:get_historical_data:64 - Fetching historical OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2023-06-30
2025-08-02 15

✅ Optimization completed for rsi
Best parameters: {'parameters': {'period': 10, 'oversold': 20, 'overbought': 80}, 'metrics': {'total_return': 0.01698873391131661, 'annualized_return': 0.06407846613294033, 'sharpe_ratio': 0.8492914204353925, 'max_drawdown': -0.008221417646134233, 'volatility': 0.027381078536949592, 'win_rate': 0.0, 'profit_factor': 0.0, 'total_trades': 0, 'final_portfolio_value': 101698.87339113167, 'current_drawdown': 0.0, 'portfolio_volatility': 0.027381078536949592}, 'rank': 1}
Best sharpe_ratio: N/A

--- Optimizing MACD ---


2025-08-02 15:13:37.923 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: macd
2025-08-02 15:13:37.923 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: macd
2025-08-02 15:13:37.923 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: macd
2025-08-02 15:13:37.923 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: macd
2025-08-02 15:13:37.924 | INFO     | src.strategies.macd_strategy:__init__:54 - MACD Strategy initialized: fast=8, slow=26, signal=15
2025-08-02 15:13:37.924 | INFO     | src.strategies.macd_strategy:__init__:54 - MACD Strategy initialized: fast=20, slow=35, signal=12
2025-08-02 15:13:37.924 | INFO     | src.strategies.macd_strategy:__init__:54 - MACD Strategy initialized: fast=12, slow=20, signal=7
2025-08-02 15:13:37.924 | INFO     | src.strategies.macd_strategy:__init__:54 - MACD Strategy initialized: fast=12, slow=26, signal=12
2025-08-02 15:13:37.924 | INFO    

✅ Optimization completed for macd
Best parameters: {'parameters': {'fast_period': 10, 'slow_period': 26, 'signal_period': 12}, 'metrics': {'total_return': 0.02135023981219253, 'annualized_return': 0.08100050407494463, 'sharpe_ratio': 1.1431964449513792, 'max_drawdown': -0.011417955747370097, 'volatility': 0.029935552334925734, 'win_rate': 0.0, 'profit_factor': 0.0, 'total_trades': 0, 'final_portfolio_value': 102135.02398121926, 'current_drawdown': -0.002396054415678094, 'portfolio_volatility': 0.029935552334925734}, 'rank': 1}
Best sharpe_ratio: N/A


In [6]:
# Cell 6: Hiển thị kết quả optimization
print("\n=== KẾT QUẢ OPTIMIZATION ===")

for strategy_type, result in optimization_results.items():
    print(f"\n{strategy_type.upper()}:")
    print(f"  Best parameters: {result.get('best_parameters', {})}")
    print(f"  Best {optimization_metric}: {result.get('best_metric_value', 'N/A')}")
    print(f"  Total combinations tested: {result.get('tested_combinations', 0)}")
    
    # Hiển thị top 5 parameters
    top_params = optimizer.get_top_parameters(strategy_type, top_n=5, metric=optimization_metric)
    print(f"  Top 5 parameters:")
    for i, params in enumerate(top_params, 1):
        print(f"    {i}. {params}")


=== KẾT QUẢ OPTIMIZATION ===

SMA_CROSSOVER:
  Best parameters: {'parameters': {'short_window': 10, 'long_window': 60}, 'metrics': {'total_return': 0.01583959644997357, 'annualized_return': 0.05965229884864942, 'sharpe_ratio': 0.7266163316764024, 'max_drawdown': -0.012492581237808715, 'volatility': 0.028069029884101097, 'win_rate': 0.0, 'profit_factor': 0.0, 'total_trades': 0, 'final_portfolio_value': 101583.95964499735, 'current_drawdown': -0.011239262286793294, 'portfolio_volatility': 0.028069029884101097}, 'rank': 1}
  Best sharpe_ratio: N/A
  Total combinations tested: 48
  Top 5 parameters:
    1. {'rank': 1, 'parameters': {'short_window': 10, 'long_window': 60}, 'metrics': {'total_return': 0.01583959644997357, 'annualized_return': 0.05965229884864942, 'sharpe_ratio': 0.7266163316764024, 'max_drawdown': -0.012492581237808715, 'volatility': 0.028069029884101097, 'win_rate': 0.0, 'profit_factor': 0.0, 'total_trades': 0, 'final_portfolio_value': 101583.95964499735, 'current_drawdown

In [7]:
# Cell 7: Tạo strategies với optimized parameters
print("\n=== TẠO STRATEGIES VỚI OPTIMIZED PARAMETERS ===")

# Xóa strategies cũ nếu có
engine.strategies = {}

# Thêm strategies với optimized parameters
for strategy_type in strategy_types:
    if strategy_type in optimization_results:
        best_params = optimization_results[strategy_type]['best_parameters']
        
        if strategy_type == 'sma_crossover':
            strategy = SMACrossoverStrategy({
                "short_window": best_params.get('short_window', 5),
                "long_window": best_params.get('long_window', 30),
                "name": f"Optimized_SMA_Crossover"
            })
            engine.add_strategy(strategy)
            print(f"✅ Added optimized SMA Crossover: {best_params}")
            
        elif strategy_type == 'rsi':
            strategy = RSIStrategy({
                "period": best_params.get('period', 14),
                "oversold": best_params.get('oversold', 30),
                "overbought": best_params.get('overbought', 70),
                "name": f"Optimized_RSI_Strategy"
            })
            engine.add_strategy(strategy)
            print(f"✅ Added optimized RSI Strategy: {best_params}")
            
        elif strategy_type == 'macd':
            strategy = MACDStrategy({
                "fast_period": best_params.get('fast_period', 15),
                "slow_period": best_params.get('slow_period', 20),
                "signal_period": best_params.get('signal_period', 7),
                "name": f"Optimized_MACD_Strategy"
            })
            engine.add_strategy(strategy)
            print(f"✅ Added optimized MACD Strategy: {best_params}")

print(f"\nTổng cộng đã thêm {len(engine.strategies)} optimized strategies")

2025-08-02 15:14:44.582 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: sma_crossover
2025-08-02 15:14:44.582 | INFO     | src.strategies.sma_crossover:__init__:48 - SMA Crossover Strategy initialized: 5/30
2025-08-02 15:14:44.582 | INFO     | src.core.trading_engine:add_strategy:82 - Added strategy: sma_crossover
2025-08-02 15:14:44.583 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: rsi
2025-08-02 15:14:44.583 | INFO     | src.strategies.rsi_strategy:__init__:56 - RSI Strategy initialized: period=14, overbought=70, oversold=30
2025-08-02 15:14:44.583 | INFO     | src.core.trading_engine:add_strategy:82 - Added strategy: rsi
2025-08-02 15:14:44.583 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: macd
2025-08-02 15:14:44.583 | INFO     | src.strategies.macd_strategy:__init__:54 - MACD Strategy initialized: fast=15, slow=20, signal=7
2025-08-02 15:14:44.584 | INFO     | src.core.trading_engine:add_


=== TẠO STRATEGIES VỚI OPTIMIZED PARAMETERS ===
✅ Added optimized SMA Crossover: {'parameters': {'short_window': 10, 'long_window': 60}, 'metrics': {'total_return': 0.01583959644997357, 'annualized_return': 0.05965229884864942, 'sharpe_ratio': 0.7266163316764024, 'max_drawdown': -0.012492581237808715, 'volatility': 0.028069029884101097, 'win_rate': 0.0, 'profit_factor': 0.0, 'total_trades': 0, 'final_portfolio_value': 101583.95964499735, 'current_drawdown': -0.011239262286793294, 'portfolio_volatility': 0.028069029884101097}, 'rank': 1}
✅ Added optimized RSI Strategy: {'parameters': {'period': 10, 'oversold': 20, 'overbought': 80}, 'metrics': {'total_return': 0.01698873391131661, 'annualized_return': 0.06407846613294033, 'sharpe_ratio': 0.8492914204353925, 'max_drawdown': -0.008221417646134233, 'volatility': 0.027381078536949592, 'win_rate': 0.0, 'profit_factor': 0.0, 'total_trades': 0, 'final_portfolio_value': 101698.87339113167, 'current_drawdown': 0.0, 'portfolio_volatility': 0.027

In [8]:
# Cell 8: Chạy backtest với optimized strategies
print("\n=== CHẠY BACKTEST VỚI OPTIMIZED STRATEGIES ===")

# Chạy backtest với optimized strategies
engine.run_backtest(backtest_start_date, backtest_end_date)

2025-08-02 15:14:57.290 | INFO     | src.core.trading_engine:run_backtest:86 - Starting backtest from 2023-07-01 to 2023-12-31
2025-08-02 15:14:57.291 | INFO     | src.data.data_manager:get_historical_data_standardized:589 - Fetching standardized OHLCV data for ['Bitstamp:BTCUSD'] from 2023-07-01 to 2023-12-31
2025-08-02 15:14:57.292 | INFO     | src.data.data_manager:get_historical_data:64 - Fetching historical OHLCV data for ['Bitstamp:BTCUSD'] from 2023-07-01 to 2023-12-31



=== CHẠY BACKTEST VỚI OPTIMIZED STRATEGIES ===


2025-08-02 15:14:57.652 | INFO     | src.data.data_manager:_fetch_tradingview_ohlcv_data:159 - Successfully fetched OHLCV data for 1 symbols
2025-08-02 15:14:57.654 | INFO     | src.data.data_manager:get_historical_data:97 - OHLCV data cached successfully
2025-08-02 15:14:57.656 | INFO     | src.data.data_manager:ensure_ohlcv_format:500 - Data formatted for Bitstamp:BTCUSD: (100, 5)
2025-08-02 15:14:57.657 | INFO     | src.data.data_manager:get_historical_data_standardized:622 - Successfully standardized OHLCV data for 1 symbols
2025-08-02 15:14:57.662 | INFO     | src.core.trading_engine:run_backtest:107 - Loaded 100 data points for [('Bitstamp:BTCUSD', 'open'), ('Bitstamp:BTCUSD', 'high'), ('Bitstamp:BTCUSD', 'low'), ('Bitstamp:BTCUSD', 'close'), ('Bitstamp:BTCUSD', 'volume')]
2025-08-02 15:14:57.679 | INFO     | src.strategies.sma_crossover:_generate_signal_for_symbol:109 - Golden cross detected for Bitstamp:BTCUSD: 118563.60 > 118355.17
2025-08-02 15:14:57.680 | INFO     | src.core

In [9]:
# Cell 9: So sánh kết quả với default parameters
print("\n=== SO SÁNH KẾT QUẢ ===")

# Tạo engine mới với default parameters để so sánh
default_engine = TradingEngine(config_path)

# Thêm strategies với default parameters
default_sma = SMACrossoverStrategy({
    "short_window": 5,
    "long_window": 30,
    "name": "Default_SMA_Crossover"
})
default_engine.add_strategy(default_sma)

default_rsi = RSIStrategy({
    "period": 14,
    "oversold": 30,
    "overbought": 70,
    "name": "Default_RSI_Strategy"
})
default_engine.add_strategy(default_rsi)

default_macd = MACDStrategy({
    "fast_period": 15,
    "slow_period": 20,
    "signal_period": 7,
    "name": "Default_MACD_Strategy"
})
default_engine.add_strategy(default_macd)

# Chạy backtest với default parameters
default_engine.run_backtest(backtest_start_date, backtest_end_date)

# Lấy kết quả
optimized_summary = engine.get_portfolio_summary()
default_summary = default_engine.get_portfolio_summary()

print("KẾT QUẢ SO SÁNH:")
print(f"{'Metric':<20} {'Optimized':<15} {'Default':<15} {'Improvement':<15}")
print("-" * 65)
print(f"{'Total Return':<20} {optimized_summary['total_return']:<15.2%} {default_summary['total_return']:<15.2%} {(optimized_summary['total_return'] - default_summary['total_return']):<15.2%}")
print(f"{'Sharpe Ratio':<20} {optimized_summary['sharpe_ratio']:<15.2f} {default_summary['sharpe_ratio']:<15.2f} {(optimized_summary['sharpe_ratio'] - default_summary['sharpe_ratio']):<15.2f}")
print(f"{'Max Drawdown':<20} {optimized_summary['max_drawdown']:<15.2%} {default_summary['max_drawdown']:<15.2%} {(optimized_summary['max_drawdown'] - default_summary['max_drawdown']):<15.2%}")
print(f"{'Win Rate':<20} {optimized_summary['win_rate']:<15.2%} {default_summary['win_rate']:<15.2%} {(optimized_summary['win_rate'] - default_summary['win_rate']):<15.2%}")
print(f"{'Total Trades':<20} {optimized_summary['total_trades']:<15} {default_summary['total_trades']:<15} {(optimized_summary['total_trades'] - default_summary['total_trades']):<15}")


2025-08-02 15:16:13.585 | INFO     | src.utils.config_manager:__init__:31 - Configuration loaded from ../../config/config.yaml
2025-08-02 15:16:13.585 | INFO     | src.risk.risk_manager:__init__:49 - Risk manager initialized
2025-08-02 15:16:13.586 | INFO     | src.core.trading_engine:__init__:77 - Trading engine initialized with $100,000.00 initial capital
2025-08-02 15:16:13.586 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: sma_crossover
2025-08-02 15:16:13.586 | INFO     | src.strategies.sma_crossover:__init__:48 - SMA Crossover Strategy initialized: 5/30
2025-08-02 15:16:13.587 | INFO     | src.core.trading_engine:add_strategy:82 - Added strategy: sma_crossover
2025-08-02 15:16:13.587 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: rsi
2025-08-02 15:16:13.587 | INFO     | src.strategies.rsi_strategy:__init__:56 - RSI Strategy initialized: period=14, overbought=70, oversold=30
2025-08-02 15:16:13.587 | INFO     | src.core


=== SO SÁNH KẾT QUẢ ===


2025-08-02 15:16:13.889 | INFO     | src.data.data_manager:_fetch_tradingview_ohlcv_data:159 - Successfully fetched OHLCV data for 1 symbols
2025-08-02 15:16:13.890 | INFO     | src.data.data_manager:get_historical_data:97 - OHLCV data cached successfully
2025-08-02 15:16:13.892 | INFO     | src.data.data_manager:ensure_ohlcv_format:500 - Data formatted for Bitstamp:BTCUSD: (100, 5)
2025-08-02 15:16:13.893 | INFO     | src.data.data_manager:get_historical_data_standardized:622 - Successfully standardized OHLCV data for 1 symbols
2025-08-02 15:16:13.897 | INFO     | src.core.trading_engine:run_backtest:107 - Loaded 100 data points for [('Bitstamp:BTCUSD', 'open'), ('Bitstamp:BTCUSD', 'high'), ('Bitstamp:BTCUSD', 'low'), ('Bitstamp:BTCUSD', 'close'), ('Bitstamp:BTCUSD', 'volume')]
2025-08-02 15:16:13.910 | INFO     | src.strategies.sma_crossover:_generate_signal_for_symbol:109 - Golden cross detected for Bitstamp:BTCUSD: 118563.60 > 118355.17
2025-08-02 15:16:13.910 | INFO     | src.core

KẾT QUẢ SO SÁNH:
Metric               Optimized       Default         Improvement    
-----------------------------------------------------------------
Total Return         -1.05%          -1.05%          0.00%          
Sharpe Ratio         -2.31           -2.31           0.00           
Max Drawdown         -1.14%          -1.14%          0.00%          
Win Rate             0.00%           0.00%           0.00%          
Total Trades         6               6               0              


In [10]:
# Cell 10: Lấy kết quả backtest cuối cùng (optimized)
portfolio_summary = engine.get_portfolio_summary()

print("\n=== KẾT QUẢ BACKTEST CUỐI CÙNG (OPTIMIZED) ===")
print(f"Initial Capital: ${portfolio_summary['initial_capital']:,.2f}")
print(f"Final Portfolio Value: ${portfolio_summary['total_value']:,.2f}")
print(f"Total Return: {portfolio_summary['total_return']:.2%}")
print(f"Annualized Return: {portfolio_summary['annualized_return']:.2%}")
print(f"Sharpe Ratio: {portfolio_summary['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {portfolio_summary['max_drawdown']:.2%}")
print(f"Win Rate: {portfolio_summary['win_rate']:.2%}")
print(f"Total Trades: {portfolio_summary['total_trades']}")


=== KẾT QUẢ BACKTEST CUỐI CÙNG (OPTIMIZED) ===
Initial Capital: $100,000.00
Final Portfolio Value: $98,948.39
Total Return: -1.05%
Annualized Return: -2.66%
Sharpe Ratio: -2.31
Max Drawdown: -1.14%
Win Rate: 0.00%
Total Trades: 6


In [11]:
# Cell 11: Visualize portfolio performance
def plot_portfolio_performance(engine):
    """Plot portfolio performance"""
    portfolio_history = engine.portfolio_history
    
    if not portfolio_history:
        print("Không có dữ liệu portfolio history")
        return
    
    df = pd.DataFrame(portfolio_history)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    # Create subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=('Portfolio Value', 'Daily Returns', 'Cumulative Returns', 
                       'Drawdown', 'Cash vs Positions', 'Trade Distribution'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Portfolio Value
    fig.add_trace(
        go.Scatter(x=df.index, y=df['total_value'], 
                  mode='lines', name='Portfolio Value'),
        row=1, col=1
    )
    
    # Daily Returns
    daily_returns = df['total_value'].pct_change()
    fig.add_trace(
        go.Scatter(x=df.index, y=daily_returns, 
                  mode='lines', name='Daily Returns'),
        row=1, col=2
    )
    
    # Cumulative Returns
    cumulative_returns = (1 + daily_returns).cumprod()
    fig.add_trace(
        go.Scatter(x=df.index, y=cumulative_returns, 
                  mode='lines', name='Cumulative Returns'),
        row=2, col=1
    )
    
    # Drawdown
    running_max = df['total_value'].expanding().max()
    drawdown = (df['total_value'] - running_max) / running_max
    fig.add_trace(
        go.Scatter(x=df.index, y=drawdown, 
                  mode='lines', name='Drawdown', fill='tonexty'),
        row=2, col=2
    )
    
    # Cash vs Positions
    fig.add_trace(
        go.Scatter(x=df.index, y=df['cash'], 
                  mode='lines', name='Cash'),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df['total_value'] - df['cash'], 
                  mode='lines', name='Positions'),
        row=3, col=1
    )
    
    # Trade Distribution (if available)
    if hasattr(engine, 'trades') and engine.trades:
        trade_returns = [trade.pnl for trade in engine.trades]
        fig.add_trace(
            go.Histogram(x=trade_returns, name='Trade Returns'),
            row=3, col=2
        )
    
    fig.update_layout(height=900, title_text="Portfolio Performance Analysis (Optimized)")
    fig.show()

# Chạy visualization
plot_portfolio_performance(engine)

In [12]:
# Cell 12: Hiển thị chi tiết trades
if hasattr(engine, 'trades') and engine.trades:
    print(f"\n=== CHI TIẾT TRADES ({len(engine.trades)} trades) ===")
    
    trades_df = pd.DataFrame([
        {
            'Symbol': trade.symbol,
            'Side': trade.side,
            'Quantity': trade.quantity,
            'Price': trade.price,
            'Timestamp': trade.timestamp,
            'Commission': trade.commission,
            'Strategy': trade.strategy
        }
        for trade in engine.trades
    ])
    
    print(trades_df.head(10))
    
    # Trade statistics
    print(f"\n=== TRADE STATISTICS ===")
    print(f"Total Trades: {len(engine.trades)}")
    print(f"Buy Trades: {len(trades_df[trades_df['Side'] == 'buy'])}")
    print(f"Sell Trades: {len(trades_df[trades_df['Side'] == 'sell'])}")
    print(f"Average Trade Size: {trades_df['Quantity'].mean():.2f}")
    print(f"Total Commission: ${trades_df['Commission'].sum():.2f}")
else:
    print("Không có trades nào được thực hiện")


=== CHI TIẾT TRADES (6 trades) ===
            Symbol  Side  Quantity     Price           Timestamp  Commission  \
0  Bitstamp:BTCUSD   buy  0.167583  119344.0 2025-07-22 23:00:00   20.000000   
1  Bitstamp:BTCUSD  sell  0.167583  115400.0 2025-07-25 07:00:00   19.339053   
2  Bitstamp:BTCUSD   buy  0.167959  118243.0 2025-07-27 07:00:00   19.859943   
3  Bitstamp:BTCUSD  sell  0.167959  118144.0 2025-07-30 07:00:00   19.843315   
4  Bitstamp:BTCUSD   buy  0.167797  118290.0 2025-07-31 19:00:00   19.848677   
5  Bitstamp:BTCUSD  sell  0.167797  116767.0 2025-07-31 23:00:00   19.593122   

        Strategy  
0  sma_crossover  
1  sma_crossover  
2  sma_crossover  
3  sma_crossover  
4  sma_crossover  
5  sma_crossover  

=== TRADE STATISTICS ===
Total Trades: 6
Buy Trades: 3
Sell Trades: 3
Average Trade Size: 0.17
Total Commission: $118.48


In [13]:
# Cell 13: Visualize trades
def viz(symbol, trades_df, historical_data):
    # Visualize
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.8, 0.2])
    fig.add_trace(go.Candlestick(x=historical_data[symbol].index,
                    open=historical_data[symbol].open,
                    high=historical_data[symbol].high,
                    low=historical_data[symbol].low,
                    close=historical_data[symbol].close,
                    ), row=1, col=1)
    set_ylim = (historical_data[symbol].low.min() * 0.98, historical_data[symbol].high.max() * 1.02)
    # Add buy/sell markers from trades_df to the candlestick chart (row 1)
    if 'trades_df' in locals():
        buy_trades = trades_df[trades_df['Side'] == 'buy']
        sell_trades = trades_df[trades_df['Side'] == 'sell']
        # Buy markers
        fig.add_trace(
            go.Scatter(
                x=buy_trades['Timestamp'],
                y=historical_data[symbol][historical_data.index.isin(buy_trades['Timestamp'])].low * 0.998,
                mode='markers+text',
                marker=dict(symbol='triangle-up', color='green', size=12),
                text=['Buy']*len(buy_trades),
                textposition='bottom center',
                name='Buy'
            ),
            row=1, col=1
        )
        # Sell markers
        fig.add_trace(
            go.Scatter(
                x=sell_trades['Timestamp'],
                y=historical_data[symbol][historical_data.index.isin(sell_trades['Timestamp'])].high * 1.002,
                mode='markers+text',
                marker=dict(symbol='triangle-down', color='red', size=12),
                text=['Sell']*len(sell_trades),
                textposition='top center',
                name='Sell'
            ),
            row=1, col=1
        )

    fig.add_trace(go.Bar(x=historical_data[symbol].index,
                        y=historical_data[symbol].volume,
                        ), row=2, col=1)

    fig.update_layout(title=f'{symbol}',
                    yaxis_range=(set_ylim[0], set_ylim[1]),
                    xaxis_title='Date',
                    yaxis_title='Price', 
                    height=800, width=1000)
    fig.show()

viz(symbols[0], trades_df, historical_data)

In [14]:
# Cell 14: Lưu kết quả optimization
print("\n=== LƯU KẾT QUẢ OPTIMIZATION ===")

# Tạo thư mục để lưu kết quả
results_dir = "../../data/optimization_results"
os.makedirs(results_dir, exist_ok=True)

# Lưu optimization results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
optimization_file = f"{results_dir}/optimization_results_{timestamp}.json"
optimizer.save_optimization_results(optimization_file)
print(f"✅ Đã lưu optimization results: {optimization_file}")

# Lưu backtest results
backtest_file = f"{results_dir}/backtest_results_{timestamp}.csv"
if hasattr(engine, 'trades') and engine.trades:
    trades_df.to_csv(backtest_file, index=False)
    print(f"✅ Đã lưu backtest results: {backtest_file}")

2025-08-02 15:17:55.637 | INFO     | src.optimization.optimizer:save_optimization_results:325 - Optimization results saved to ../../data/optimization_results/optimization_results_20250802_151755.json



=== LƯU KẾT QUẢ OPTIMIZATION ===
✅ Đã lưu optimization results: ../../data/optimization_results/optimization_results_20250802_151755.json
✅ Đã lưu backtest results: ../../data/optimization_results/backtest_results_20250802_151755.csv


In [17]:
# Cell 15: Hiển thị kết quả optimization một cách trực quan
print("\n=== HIỂN THỊ KẾT QUẢ OPTIMIZATION ===")

# Hiển thị kết quả optimization cho từng strategy
for strategy_type in strategy_types:
    if strategy_type in optimization_results:
        print(f"\n--- Visualization cho {strategy_type.upper()} ---")
        
        # Plot optimization results
        plot_optimization_results(optimization_results, strategy_type)
        
        # Plot parameter heatmap
        plot_parameter_heatmap(optimization_results, strategy_type)
        
        # Plot optimization progress
        plot_optimization_progress(optimization_results, strategy_type)

# Tạo báo cáo tổng hợp
print("\n--- TẠO BÁO CÁO TỔNG HỢP ---")
report_path = f"{results_dir}/optimization_report_{timestamp}.txt"
create_optimization_report(optimization_results, report_path)

print("🎉 Hoàn thành optimization và backtest!")
# Gợi ý xử lý lỗi TypeError: float() argument must be a string or a real number, not 'dict'
# Lỗi này thường xảy ra khi dữ liệu optimization_results có trường metrics là dict thay vì giá trị số.
# Để debug, hãy kiểm tra cấu trúc optimization_results cho từng strategy_type:

for strategy_type in strategy_types:
    if strategy_type in optimization_results:
        print(f"\n[DEBUG] {strategy_type} - Top 1 result:")
        best_result = optimization_results[strategy_type][0]
        print(best_result)
        print("metrics:", best_result.get("metrics"))
        print("parameters:", best_result.get("parameters"))
        # Kiểm tra các trường metrics có phải là dict lồng dict không
        if isinstance(best_result.get("metrics"), dict):
            for k, v in best_result["metrics"].items():
                print(f"  {k}: {v} ({type(v)})")

# Nếu metrics là dict lồng dict, cần flatten trước khi chuyển thành DataFrame hoặc dùng cho heatmap.



=== HIỂN THỊ KẾT QUẢ OPTIMIZATION ===

--- Visualization cho SMA_CROSSOVER ---


TypeError: float() argument must be a string or a real number, not 'dict'

In [ ]:
# Cell 4: Thêm strategies
# SMA Crossover Strategy
sma_strategy = SMACrossoverStrategy({
    "short_window": engine.config.get("strategies.sma_crossover.short_window", 5),
    "long_window": engine.config.get("strategies.sma_crossover.long_window", 30),
    "name": "SMA_Crossover"
})
engine.add_strategy(sma_strategy)

# RSI Strategy
rsi_strategy = RSIStrategy({
    "period": engine.config.get("strategies.rsi.period", 14),
    "oversold": engine.config.get("strategies.rsi.oversold", 30),
    "overbought": engine.config.get("strategies.rsi.overbought", 70),
    "name": "RSI_Strategy"
})
engine.add_strategy(rsi_strategy)

# MACD Strategy
macd_strategy = MACDStrategy({
    "fast_period": engine.config.get("strategies.macd.fast_period", 15),
    "slow_period": engine.config.get("strategies.macd.slow_period", 20),
    "signal_period": engine.config.get("strategies.macd.signal_period", 7),
    "name": "MACD_Strategy"
})
engine.add_strategy(macd_strategy)

print(f"Đã thêm {len(engine.strategies)} strategies:")
for name, strategy in engine.strategies.items():
    print(f"  - {name}: {strategy.__class__.__name__}")

2025-08-02 12:16:48.465 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: sma_crossover
2025-08-02 12:16:48.466 | INFO     | src.strategies.sma_crossover:__init__:48 - SMA Crossover Strategy initialized: 5/30
2025-08-02 12:16:48.467 | INFO     | src.core.trading_engine:add_strategy:82 - Added strategy: sma_crossover
2025-08-02 12:16:48.468 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: rsi
2025-08-02 12:16:48.469 | INFO     | src.strategies.rsi_strategy:__init__:56 - RSI Strategy initialized: period=20, overbought=70, oversold=30
2025-08-02 12:16:48.470 | INFO     | src.core.trading_engine:add_strategy:82 - Added strategy: rsi
2025-08-02 12:16:48.470 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: macd
2025-08-02 12:16:48.471 | INFO     | src.strategies.macd_strategy:__init__:54 - MACD Strategy initialized: fast=15, slow=20, signal=7
2025-08-02 12:16:48.471 | INFO     | src.core.trading_engine:add_

Đã thêm 3 strategies:
  - sma_crossover: SMACrossoverStrategy
  - rsi: RSIStrategy
  - macd: MACDStrategy


In [6]:
# Cell 5: Chạy backtest
print("Bắt đầu chạy backtest...")

# Chạy chiến lược
print('SMA Crossover:', engine.config.get('strategies.sma_crossover.enabled', False))
print('RSI:', engine.config.get('strategies.rsi.enabled', False))
print('MACD:', engine.config.get('strategies.macd.enabled', False), '\n')

# Chạy backtest với chiến lược đã chọn
engine.run_backtest(start_date, end_date)

2025-08-02 12:16:48.478 | INFO     | src.core.trading_engine:run_backtest:86 - Starting backtest from 2023-01-01 to 2025-07-31
2025-08-02 12:16:48.488 | INFO     | src.data.data_manager:get_historical_data_standardized:589 - Fetching standardized OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2025-07-31
2025-08-02 12:16:48.488 | INFO     | src.data.data_manager:get_historical_data:64 - Fetching historical OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2025-07-31


Bắt đầu chạy backtest...
SMA Crossover: True
RSI: False
MACD: False 



2025-08-02 12:16:48.924 | INFO     | src.data.data_manager:_fetch_tradingview_ohlcv_data:159 - Successfully fetched OHLCV data for 1 symbols
2025-08-02 12:16:48.928 | INFO     | src.data.data_manager:get_historical_data:97 - OHLCV data cached successfully
2025-08-02 12:16:48.932 | INFO     | src.data.data_manager:ensure_ohlcv_format:500 - Data formatted for Bitstamp:BTCUSD: (100, 5)
2025-08-02 12:16:48.934 | INFO     | src.data.data_manager:get_historical_data_standardized:622 - Successfully standardized OHLCV data for 1 symbols
2025-08-02 12:16:48.941 | INFO     | src.core.trading_engine:run_backtest:107 - Loaded 100 data points for [('Bitstamp:BTCUSD', 'open'), ('Bitstamp:BTCUSD', 'high'), ('Bitstamp:BTCUSD', 'low'), ('Bitstamp:BTCUSD', 'close'), ('Bitstamp:BTCUSD', 'volume')]
2025-08-02 12:16:48.958 | INFO     | src.strategies.sma_crossover:_generate_signal_for_symbol:109 - Golden cross detected for Bitstamp:BTCUSD: 118563.60 > 118355.17
2025-08-02 12:16:48.959 | INFO     | src.core

In [7]:
# Cell 6: Lấy kết quả backtest
portfolio_summary = engine.get_portfolio_summary()

print("=== KẾT QUẢ BACKTEST ===")
print(f"Initial Capital: ${portfolio_summary['initial_capital']:,.2f}")
print(f"Final Portfolio Value: ${portfolio_summary['total_value']:,.2f}")
print(f"Total Return: {portfolio_summary['total_return']:.2%}")
print(f"Annualized Return: {portfolio_summary['annualized_return']:.2%}")
print(f"Sharpe Ratio: {portfolio_summary['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {portfolio_summary['max_drawdown']:.2%}")
print(f"Win Rate: {portfolio_summary['win_rate']:.2%}")
print(f"Total Trades: {portfolio_summary['total_trades']}")

=== KẾT QUẢ BACKTEST ===
Initial Capital: $100,000.00
Final Portfolio Value: $98,948.39
Total Return: -1.05%
Annualized Return: -2.66%
Sharpe Ratio: -2.31
Max Drawdown: -1.14%
Win Rate: 0.00%
Total Trades: 6


In [8]:
# Cell 7: Visualize portfolio performance
def plot_portfolio_performance(engine):
    """Plot portfolio performance"""
    portfolio_history = engine.portfolio_history
    
    if not portfolio_history:
        print("Không có dữ liệu portfolio history")
        return
    
    df = pd.DataFrame(portfolio_history)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    # Create subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=('Portfolio Value', 'Daily Returns', 'Cumulative Returns', 
                       'Drawdown', 'Cash vs Positions', 'Trade Distribution'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Portfolio Value
    fig.add_trace(
        go.Scatter(x=df.index, y=df['total_value'], 
                  mode='lines', name='Portfolio Value'),
        row=1, col=1
    )
    
    # Daily Returns
    daily_returns = df['total_value'].pct_change()
    fig.add_trace(
        go.Scatter(x=df.index, y=daily_returns, 
                  mode='lines', name='Daily Returns'),
        row=1, col=2
    )
    
    # Cumulative Returns
    cumulative_returns = (1 + daily_returns).cumprod()
    fig.add_trace(
        go.Scatter(x=df.index, y=cumulative_returns, 
                  mode='lines', name='Cumulative Returns'),
        row=2, col=1
    )
    
    # Drawdown
    running_max = df['total_value'].expanding().max()
    drawdown = (df['total_value'] - running_max) / running_max
    fig.add_trace(
        go.Scatter(x=df.index, y=drawdown, 
                  mode='lines', name='Drawdown', fill='tonexty'),
        row=2, col=2
    )
    
    # Cash vs Positions
    fig.add_trace(
        go.Scatter(x=df.index, y=df['cash'], 
                  mode='lines', name='Cash'),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df['total_value'] - df['cash'], 
                  mode='lines', name='Positions'),
        row=3, col=1
    )
    
    # Trade Distribution (if available)
    if hasattr(engine, 'trades') and engine.trades:
        trade_returns = [trade.pnl for trade in engine.trades]
        fig.add_trace(
            go.Histogram(x=trade_returns, name='Trade Returns'),
            row=3, col=2
        )
    
    fig.update_layout(height=900, title_text="Portfolio Performance Analysis")
    fig.show()

# Chạy visualization
plot_portfolio_performance(engine)

In [9]:
# Cell 8: Hiển thị chi tiết trades
if hasattr(engine, 'trades') and engine.trades:
    print(f"\n=== CHI TIẾT TRADES ({len(engine.trades)} trades) ===")
    
    trades_df = pd.DataFrame([
        {
            'Symbol': trade.symbol,
            'Side': trade.side,
            'Quantity': trade.quantity,
            'Price': trade.price,
            'Timestamp': trade.timestamp,
            'Commission': trade.commission,
            'Strategy': trade.strategy
        }
        for trade in engine.trades
    ])
    
    print(trades_df.head(10))
    
    # Trade statistics
    print(f"\n=== TRADE STATISTICS ===")
    print(f"Total Trades: {len(engine.trades)}")
    print(f"Buy Trades: {len(trades_df[trades_df['Side'] == 'buy'])}")
    print(f"Sell Trades: {len(trades_df[trades_df['Side'] == 'sell'])}")
    print(f"Average Trade Size: {trades_df['Quantity'].mean():.2f}")
    print(f"Total Commission: ${trades_df['Commission'].sum():.2f}")
else:
    print("Không có trades nào được thực hiện")


=== CHI TIẾT TRADES (6 trades) ===
            Symbol  Side  Quantity     Price           Timestamp  Commission  \
0  Bitstamp:BTCUSD   buy  0.167583  119344.0 2025-07-22 23:00:00   20.000000   
1  Bitstamp:BTCUSD  sell  0.167583  115400.0 2025-07-25 07:00:00   19.339053   
2  Bitstamp:BTCUSD   buy  0.167959  118243.0 2025-07-27 07:00:00   19.859943   
3  Bitstamp:BTCUSD  sell  0.167959  118144.0 2025-07-30 07:00:00   19.843315   
4  Bitstamp:BTCUSD   buy  0.167797  118290.0 2025-07-31 19:00:00   19.848677   
5  Bitstamp:BTCUSD  sell  0.167797  116767.0 2025-07-31 23:00:00   19.593122   

        Strategy  
0  sma_crossover  
1  sma_crossover  
2  sma_crossover  
3  sma_crossover  
4  sma_crossover  
5  sma_crossover  

=== TRADE STATISTICS ===
Total Trades: 6
Buy Trades: 3
Sell Trades: 3
Average Trade Size: 0.17
Total Commission: $118.48


In [10]:
def viz(symbol, trades_df, historical_data):
    # Visualize
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.8, 0.2])
    fig.add_trace(go.Candlestick(x=historical_data[symbol].index,
                    open=historical_data[symbol].open,
                    high=historical_data[symbol].high,
                    low=historical_data[symbol].low,
                    close=historical_data[symbol].close,
                    ), row=1, col=1)
    set_ylim = (historical_data[symbol].low.min() * 0.98, historical_data[symbol].high.max() * 1.02)
    # Add buy/sell markers from trades_df to the candlestick chart (row 1)
    if 'trades_df' in locals():
        buy_trades = trades_df[trades_df['Side'] == 'buy']
        sell_trades = trades_df[trades_df['Side'] == 'sell']
        # Buy markers
        fig.add_trace(
            go.Scatter(
                x=buy_trades['Timestamp'],
                y=historical_data[symbol][historical_data.index.isin(buy_trades['Timestamp'])].low * 0.998,
                mode='markers+text',
                marker=dict(symbol='triangle-up', color='green', size=12),
                text=['Buy']*len(buy_trades),
                textposition='bottom center',
                name='Buy'
            ),
            row=1, col=1
        )
        # Sell markers
        fig.add_trace(
            go.Scatter(
                x=sell_trades['Timestamp'],
                y=historical_data[symbol][historical_data.index.isin(sell_trades['Timestamp'])].high * 1.002,
                mode='markers+text',
                marker=dict(symbol='triangle-down', color='red', size=12),
                text=['Sell']*len(sell_trades),
                textposition='top center',
                name='Sell'
            ),
            row=1, col=1
        )

    fig.add_trace(go.Bar(x=historical_data[symbol].index,
                        y=historical_data[symbol].volume,
                        ), row=2, col=1)

    fig.update_layout(title=f'{symbol}',
                    yaxis_range=(set_ylim[0], set_ylim[1]),
                    xaxis_title='Date',
                    yaxis_title='Price', 
                    height=800, width=1000)
    fig.show()

viz(symbols[0], trades_df, historical_data)

In [11]:
# Cell 9: So sánh performance của các strategies
def compare_strategies(engine):
    """Compare performance of different strategies"""
    if not hasattr(engine, 'strategies') or not engine.strategies:
        print("Không có strategies để so sánh")
        return
    
    strategy_results = {}
    
    for name, strategy in engine.strategies.items():
        # Calculate basic metrics for each strategy
        # This is a simplified version - you might want to run separate backtests
        strategy_results[name] = {
            'name': name,
            'type': strategy.__class__.__name__,
            'parameters': strategy.get_summary()
        }
    
    # Display strategy comparison
    print("=== STRATEGY COMPARISON ===")
    for name, result in strategy_results.items():
        print(f"\nStrategy: {name}")
        print(f"Type: {result['type']}")
        print(f"Parameters: {result['parameters']}")

compare_strategies(engine)

=== STRATEGY COMPARISON ===

Strategy: sma_crossover
Type: SMACrossoverStrategy
Parameters: {'name': 'sma_crossover', 'strategy_type': 'SMA Crossover', 'short_window': 5, 'long_window': 30, 'description': 'SMA Crossover with 5/30 periods'}

Strategy: rsi
Type: RSIStrategy
Parameters: {'name': 'rsi', 'strategy_type': 'RSI Strategy', 'period': 20, 'overbought_threshold': 70, 'oversold_threshold': 30, 'confirmation_period': 2, 'description': 'RSI Strategy with 20 period, 30/70 levels'}

Strategy: macd
Type: MACDStrategy
Parameters: {'name': 'macd', 'strategy_type': 'MACD Strategy', 'fast_period': 15, 'slow_period': 20, 'signal_period': 7, 'histogram_threshold': 0.0, 'confirmation_period': 1, 'description': 'MACD Strategy with 15/20/7 periods'}
